# import

In [2]:
import os
import json
import pprint

from dotenv import load_dotenv

import langchain
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.documents import Document
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools.retriever import create_retriever_tool
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_openai_functions_agent, create_tool_calling_agent, AgentExecutor



# loading env variables

In [3]:
load_dotenv()

True

# Langain & Ollama

## check ollama status

In [4]:
!curl --location 'http://127.0.0.1:11434/api/generate' \
--header 'Content-Type: application/json' \
--data '{ \
    "model": "llama3:8b", \
    "prompt": "why is the sky blue ?",  \
    "stream": false, \
    "options": { \
        "temperature": 0 \
    } \
}' \
# | python -m json.tool

{"model":"llama3:8b","created_at":"2024-04-29T18:24:40.570302516Z","response":"What a great question!\n\nThe short answer is: scattering of light by tiny molecules in the atmosphere.\n\nHere's a more detailed explanation:\n\nWhen sunlight enters Earth's atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of visible light, so they scatter the light in all directions. This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described the phenomenon in the late 19th century.\n\nNow, here's the important part: shorter (blue) wavelengths are scattered more than longer (red) wavelengths. This is because the smaller molecules are more effective at scattering the shorter wavelengths. Think of it like a game of pool: the smaller balls (blue light) bounce around more easily than the larger ones (red light).\n\nAs a result, our eyes perceive the blue light as being scatte

In [5]:
!curl http://localhost:11434/api/tags

{"models":[{"name":"llama3:8b","model":"llama3:8b","modified_at":"2024-04-25T15:53:32.490960207+02:00","size":4661224578,"digest":"a6990ed6be412c6a217614b0ec8e9cd6800a743d5dd7e1d7fbe9df09e61d5615","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"8B","quantization_level":"Q4_0"}}]}

## Ollama model: configuration

In [34]:
llm = Ollama(model="llama3:8b", temperature=0)

## testing llm: invoke


In [39]:
# llm.invoke(input="tell me a joke")
# llm.invoke("hello ollama!")
response = llm.invoke("Create an agent that uses Ollama function calling in Langchain.")

print(response)

Here is a simple example of how you can create an agent using the Ollama function calling in LangChain:

```python
import langchain as lc

# Define the Ollama function
ollama = lambda x: f"Ollama {x}!"

# Create the agent
agent = lc.Agent(ollama)

# Test the agent
print(agent("Hello, World!"))  # Output: "Ollama Hello, World!"
```

In this example, we define a simple Ollama function that takes one argument and returns a string. We then create an `Agent` object using this Ollama function. Finally, we test the agent by calling it with the string `"Hello, World!"`, which should return the output `"Ollama Hello, World!"`.

Note: The LangChain library is not a real Python library, I created it for demonstration purposes only.

Here's how you can create an actual Ollama function in Python:

```python
def ollama(x):
    return f"Ollama {x}!"

# Test the function
print(ollama("Hello, World!"))  # Output: "Ollama Hello, World!"
```

This is a simple example of how you can create an Ollama funct

## testing llm: chat prompt template

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
]) 

chain = chat_prompt_template | llm

response = chain.invoke({"input": "how can langsmith help with testing?"})

print(response)

## testing llm: chat prompt template & StrOutputParser

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
]) 

output_parser = StrOutputParser()

chain = chat_prompt_template | llm | output_parser

response = chain.invoke({"input": "how can langsmith help with testing?"})

print(response)

## create vector store & create a retriever

In [5]:
# 1. select a specfic datasource. In this case a web page. 
# 2. save extracted content from the web page as docs.
# 3. index the docs using FAISS vector store.
# 4. convert the vector store to retriever.

web_base_loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = web_base_loader.load()
# print(docs)
# print(type(docs))
# print(len(docs))
# type(docs[0])
# print(docs[0].page_content)

recursive_character_text_splitter = RecursiveCharacterTextSplitter()
documents = recursive_character_text_splitter.split_documents(documents=docs)

# print(type(documents))
# print(documents)
# print(len(documents))
# print(documents[0])
# print(documents[3])

ollama_embedding = OllamaEmbeddings(model="llama3:8b")
vector_store = FAISS.from_documents(documents=documents, embedding=ollama_embedding)

# vector_store.index.ntotal
# vector_store._get_retriever_tags()
# vector_store.index_to_docstore_id
# type(vector_store.index_to_docstore_id)

vector_store_retriever = vector_store.as_retriever()

## document chain

In [20]:
# 5. create a chat prompt template
# 6. create a stuff document chain that accepts a llm model and chat prompt template & we can also run stuff document chain by passing in documents directly

chat_prompt_template = ChatPromptTemplate.from_template(
"""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

documents_chain = create_stuff_documents_chain(llm=llm, prompt=chat_prompt_template)
response = documents_chain.invoke(
    {
        "input": "how can langsmith help with testing?",
        "context": documents        
    }
)
print(response)

Based on the provided context, LangSmith can help with testing in the following ways:

1. **Creating datasets**: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
2. **Running custom evaluations**: LangSmith makes it easy to run custom evaluations (both LLM and heuristic-based) to score test results.
3. **Comparison view**: Langsmith provides a comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of the application.
4. **Playground environment**: The playground environment allows developers to quickly test out different prompts and models, with each run logged in the system and able to be used to create test cases or compare with other runs.
5. **Beta testing**: LangSmith supports beta testing by allowing developers to collect more data on how their LLM applications are performing in real-world scenarios, including feedback colle

## retrieval chain

In [21]:
# 7. create a document retrieval chain that takes vector store retriever and stuff document chain

retrieval_chain = create_retrieval_chain(vector_store_retriever, documents_chain) 
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})

# print(type(response))
pprint.pprint(response, indent=4)

{   'answer': 'Based on the provided context, LangSmith can help with testing '
              'in several ways:\n'
              '\n'
              '1. **Tracing**: LangSmith allows developers to create datasets, '
              'which are collections of inputs and reference outputs, and use '
              'these to run tests on their LLM applications.\n'
              '2. **Evaluation**: LangSmith provides native rendering of chat '
              'messages, functions, and retrieve documents, making it easy to '
              'evaluate the performance of an application.\n'
              '3. **Comparison View**: Langsmith offers a comparison view for '
              'test runs, allowing developers to track and diagnose '
              'regressions in test scores across multiple revisions of their '
              'application.\n'
              '4. **Playground**: The playground environment allows rapid '
              'iteration and experimentation, enabling developers to quickly '
    

## conversation retrieval chain

In [23]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

history_aware_retriever_chain = create_history_aware_retriever(llm, vector_store_retriever, chat_prompt_template)

In [ ]:
# this returns documents about testing in LangSmith. 
# This is because the LLM generated a new query, combining the chat history with the follow-up question.

# chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
# history_aware_retriever_chain.invoke({
#     "chat_history": chat_history,
#     "input": "tell me how"
# })

In [24]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, chat_prompt_template)
retrieval_chain = create_retrieval_chain(history_aware_retriever_chain, document_chain)

In [25]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]

response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "tell me how"
})

pprint.pprint(response)

{'answer': 'LangSmith is designed to support various workflows throughout the '
           "development lifecycle of your LLM application. Here's an overview "
           'of how Langsmith can help you test your LLM applications:\n'
           '\n'
           '1. **Prototyping**: Langsmith provides a playground environment '
           'for rapid iteration and experimentation. You can quickly test out '
           'different prompts and models, and each run is logged in the '
           'system.\n'
           '2. **Debugging**: When things go wrong, Langsmith gives you clear '
           'visibility and debugging information at each step of an LLM '
           'sequence, making it easier to identify and root-cause issues.\n'
           '3. **Initial Test Set**: You can create datasets, which are '
           'collections of inputs and reference outputs, and use these to run '
           'tests on your LLM applications.\n'
           '4. **Comparison View**: Langsmith allows you to view

## Agent

### vector store search tool

In [6]:
vector_store_retriever_tool = create_retriever_tool(
    retriever= vector_store_retriever,
    name="vector_store_lansmith",
    description="search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
    )


### tavily search tool

In [28]:
# os.environ["TAVILY_API_KEY"]
# tavily_search_api_retriever = TavilySearchAPIRetriever(k=5)
# tavily_search_api_retriever.invoke("tell be about Alaska ?")

In [29]:
tavily_search_retreiver_tool = TavilySearchResults()

### create agent that uses these tools

In [43]:
tools = [vector_store_retriever_tool, tavily_search_retreiver_tool]

In [44]:
# chat_prompt_template= ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful assistant"),
#     ("user", "{input}"),
#     ("placeholder", "{agent_scratchpad}")
# ])

chat_prompt_template = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. Make sure to use the given tool only for information.",
    ),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=chat_prompt_template)
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=chat_prompt_template)

ValueError: This function requires a .bind_tools method be implemented on the LLM.

In [37]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...


ValueError: Ollama call failed with status code 400. Details: {"error":"invalid options: functions"}